<a href="https://colab.research.google.com/github/miguelvelascop/ems2022_lab_2_git_iwt31-grupoiwt31_09/blob/master/PreprocesadoDatasetDepresjon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importo librerías para manejar vectores y matrices (numpy) y manejar tablas (panda) más facilmente, y os, que permite manejarte por el sistema de archivos más facilmente(en este caso por GoogleDrive).

In [ ]:
import numpy as np
import pandas as pd
import os

También importo las funciones de las librería de colab que me permiten cargar archivos.

In [ ]:
from google.colab import files, drive

Vamos creando dataframes con cada csv cargado, le aplicamos las funciones para obtener la media, desviación típica, el mínimo y el máximo, lo pasamos a arrays numpy para poder ir uniendolos y conseguir al final una matriz numpy con todos los datos necesarios, y las etiquetas añadidas. Repetimos el proceso tanto con archivos de depresión como no depresión, y al final concateno ambas matrices.

In [ ]:
df = pd.DataFrame()
#Cargamos tres listas con todos los archivos csv del dataset (una para no depresión, otra para depresión y otro con datos sobre lso voluntarios)
ruta_Depresion='/content/drive/MyDrive/TFG/datasets/depresjon/condition'
ruta_noDepresion='/content/drive/MyDrive/TFG/datasets/depresjon/control'
ruta_Scores='/content/drive/MyDrive/TFG/datasets/depresjon/scores.csv'
archivos_De = os.listdir(ruta_Depresion)
archivos_noDe = os.listdir(ruta_noDepresion)
scores=pd.read_csv(ruta_Scores,  sep=',',  comment='#')

#Declaramos una función para filtrar los datos incompletos posteriormente
dayIsComplete = lambda x: x['activity'].count() == 1440

#Agrupo todos los datos de depresión en un pandas
#Prepocesado datos depresión

cont=0;
for i in archivos_De:
  aux=pd.read_csv(ruta_Depresion+'/'+i,  sep=',',  comment='#')
  aux['timestamp'] = pd.to_datetime(aux['timestamp'])

  aux = aux.groupby('date').filter(dayIsComplete)

  media = aux.groupby('date')['activity'].mean().to_numpy()
  matriz=media

  desv = aux.groupby('date')['activity'].std().to_numpy()
  matriz = np.append([matriz],[desv],axis=0)

  min = aux.groupby('date')['activity'].min().to_numpy()
  matriz2 = min

  max = aux.groupby('date')['activity'].max().to_numpy()
  matriz2 = np.append([matriz2],[max],axis=0)

  matriz = np.insert(matriz, matriz.shape[0], matriz2, 0)
  total = aux.groupby('date')['activity'].sum().to_numpy()
  matriz = np.append(matriz,[total],axis=0)

  matriz=np.transpose(matriz)

  #Añadir columna con genero
  selected_score = scores[scores['number'] == i.split('.')[0]]
  gender = selected_score['gender'].values[0]
  n_filas = matriz.shape[0]
  columna_genero = np.full((n_filas, 1), gender)
  matriz = np.hstack((matriz, columna_genero))

  if(cont==0):
    matrizdefD=matriz
  else:
    matrizdefD=np.concatenate((matrizdefD, matriz))
  cont=cont+1

mediaAuxD = matrizdefD[:,4].mean()
desED = matrizdefD[:,4].std()

#Añado fila de etiqueta (0, Depresión)
a = np.zeros((1,matrizdefD.shape[0]))
matrizdefD = np.insert(matrizdefD, 6, a, 1)

print("Depession data size:")
print(matrizdefD.shape)

#######################################
#Prepocesado datos no depresión

cont=0
for i in archivos_noDe:
  aux=pd.read_csv(ruta_noDepresion+'/'+i,  sep=',',  comment='#')
  aux['timestamp'] = pd.to_datetime(aux['timestamp'])

  aux = aux.groupby('date').filter(dayIsComplete)

  media = aux.groupby('date')['activity'].mean().to_numpy()
  matriz=media

  desv = aux.groupby('date')['activity'].std().to_numpy()
  matriz = np.append([matriz],[desv],axis=0)

  min = aux.groupby('date')['activity'].min().to_numpy()
  matriz2 = min

  max = aux.groupby('date')['activity'].max().to_numpy()
  matriz2 = np.append([matriz2],[max],axis=0)

  matriz = np.insert(matriz, matriz.shape[0], matriz2, 0)
  total = aux.groupby('date')['activity'].sum().to_numpy()
  matriz = np.append(matriz,[total],axis=0)

  matriz=np.transpose(matriz)

  #Añadir columna con genero
  selected_score = scores[scores['number'] == i.split('.')[0]]
  gender = selected_score['gender'].values[0]
  n_filas = matriz.shape[0]
  columna_genero = np.full((n_filas, 1), gender)
  matriz = np.hstack((matriz, columna_genero))


  if(cont==0):
    matrizdefND=matriz
  else:
    matrizdefND=np.concatenate((matrizdefND, matriz))
  cont=cont+1

mediaAuxND = matrizdefND[:,4].mean()
desEND = matrizdefND[:,4].std()

#Añado fila de etiqueta (0, Depresión)
a = np.ones((1,matrizdefND.shape[0]))
matrizdefND = np.insert(matrizdefND, 6, a, 1)

print("No depession data size:")
print(matrizdefND.shape)
#Concatenar matrizes de depresión y no depresión

matrizDef=np.concatenate((matrizdefD, matrizdefND))

print("Final data size:")
print(matrizDef.shape)



237714.25905292478 154473.72459996314
Depession data:
[1.16794444e+02 2.33477859e+02 0.00000000e+00 1.77300000e+03
 1.68184000e+05 2.00000000e+00 0.00000000e+00]
Depession data size:
(359, 7)
270335.8805970149 219628.71432567041
No depession data:
[1.85568056e+02 3.46555786e+02 0.00000000e+00 3.09700000e+03
 2.67218000e+05 2.00000000e+00 1.00000000e+00]
No depession data size:
(670, 7)
Final data size:
(1029, 7)


Ahora pasamos el numpy con los datos obtenidos a un dataframe pandas

In [ ]:
dataFrame= pd.DataFrame(matrizDef, columns = ['media','desviacion_tipica','minimo','maximo','total','genero','etiqueta'])

,media,desviacion_tipica,minimo,maximo,total,genero,etiqueta
0,116.794444,233.477859,0.0,1773.0,168184.0,2.0,0.0
1,257.561111,386.187859,0.0,3103.0,370888.0,2.0,0.0
2,195.871528,296.758489,0.0,1946.0,282055.0,2.0,0.0
3,168.595139,267.342964,0.0,1886.0,242777.0,2.0,0.0
4,174.361806,299.574691,0.0,3622.0,251081.0,2.0,0.0
...,...,...,...,...,...,...,...
1024,2.460417,1.152615,0.0,3.0,3543.0,1.0,1.0
1025,2.543750,1.077679,0.0,3.0,3663.0,1.0,1.0
1026,1.745833,1.480232,0.0,3.0,2514.0,1.0,1.0
1027,1.252083,1.479885,0.0,3.0,1803.0,1.0,1.0


Por último, importamos el dataframe a un csv y lo descargamos

In [ ]:
#Con index=False quitamos los índices
dataFrame.to_csv('Dataset_Depresjon_procesado.csv', index=False)
files.download('Dataset_Depresjon_procesado.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>